# Project 2: Investigating borrower reliability

## Introduction

The client is a bank's credit department, and the objective is to determine whether a client's marital status and the number of children they have impact the timely repayment of loans. The bank provides input data consisting of clients' creditworthiness statistics.

The results of this study will be incorporated into the development of a credit scoring model—a specialized system that evaluates a potential borrower's ability to repay a loan to the bank.

**Research goals** - to test four hypotheses:

1. Is there a correlation between having children and repaying a loan on time?
2. Is there a correlation between marital status and repaying a loan on time?
3. Is there a correlation between income level and repaying a loan on time?
4. How do different loan purposes affect the timely repayment of loans?

**Data description**

The table consists of 12 columns:

1. `children` — the number of children in the family
2. `days_employed` — total employment history in days
3. `dob_years` — client's age in years
4. `education` — client's education level
5. `education_id` — identifier for the education level
6. `family_status` — marital status
7. `family_status_id` — identifier for the marital status
8. `gender` — client's gender
9. `income_type` — type of employment
10. `debt` — whether the client had debt related to loan repayment (yes/no)
11. `total_income` — monthly income
12. `purpose` — purpose of the loan

Each row in the dataset represents a client. The data includes both categorical variables (such as education, income_type) and quantitative variables (like total_income, children). The data appears sufficient for testing the stated hypotheses.

In the subsequent analysis, we will examine if there are any artifacts in the data that require attention. For now, it is worth noting that all column names are already recorded in the snake_case format.

## Data preprocessing

Initially, we need to examine the available data. Our areas of interest include:

1. Artifacts in the data
2. Missing (None) values in columns
3. Duplicate rows, non-standard category names, and data types in columns
4. Lookup tables
5. Classification of total_income

We will address these points one-by-one.

In [3]:
import pandas as pd
import numpy as np
from IPython.display import display

try:
    raw_df = pd.read_csv('/datasets/data.csv')
except:
    raw_df = pd.read_csv('raw_credit_scoring.csv')

df = raw_df.copy()

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


The first observation is that the categorical columns have `object` data type, representing text fields. Most of the quantitative columns have `float` data type, with the exception of `children` and `dob_years`, which have `int` data type (this makes sense, as the number of children cannot be a fraction).

The second observation is that the columns have varying numbers of rows; both `days_employed` and `total_income` have approximately 2,000 fewer rows than the others. This suggests that some records (rows) are missing these values.

Lastly, it appears that the columns `education_id` and `family_status_id` contain identifiers (keys) for different statuses in the `education` and `family_status` columns.

In [6]:
display(df.head(10))

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


From the first 10 rows, it is clear that the data needs to be processed before analysis. For example, in the `education` column, the data is not standardized: a value can be `СРЕДНЕЕ` or `Среднее`. In the `days_employed` column, we also encounter unusual values: either negative or greater than 300,000.

### Artifacts in the Data

Before we perform any operations with the data (such as replacing empty values with averages), let's check for and correct any possible artifacts.

In [7]:
columns_values =['children', 'days_employed', 'dob_years', 'total_income']

columns_category = [
    'education',
    'education_id',
    'family_status',
    'family_status_id',
    'gender',
    'income_type',
    'debt',
    'purpose'
]

round(df[columns_values].describe(), 1)

,children,days_employed,dob_years,total_income
count,21525.0,19351.0,21525.0,19351.0
mean,0.5,63046.5,43.3,167422.3
std,1.4,140827.3,12.6,102971.6
min,-1.0,-18388.9,0.0,20667.3
25%,0.0,-2747.4,33.0,103053.2
50%,0.0,-1203.4,42.0,145017.9
75%,1.0,-291.1,53.0,203435.1
max,20.0,401755.4,75.0,2265604.0


We're not very fortunate: we encounter children counts of -1 and 20, applicants from nursery groups (with an age of zero), as well as negative (and millennia-long(!)) work experience.

Under normal circumstances, I would have many questions for the owner of this data. However, since this is an educational exercise, I will correct these artifacts based on a few assumptions:

1. `children = -1` and `children = 20` - likely due to some error correction logic in the system. It seems that upon input error, the database records -1. And 20 is an upper limit (also confirmed by the absence of values between 5 and 20). We will replace them with the average value for the corresponding `income_type` group. For example, a retiree is more likely to have several children, while a student has none. This assumption is a stretch for children = 20, but given the tight deadlines, I have no other options.

2. We will do the same for `dob_years`. It is unlikely that we will see 60-year-old students or 20-year-old retirees.

3. `days_employed` is slightly more complicated. We have two issues here: negative values and millennia-long experiences. We can fix the negative values with the abs() function (possibly due to data import errors). I thought that millennia might represent work experience in hours, not days. Since this column will not be used in hypothesis testing, we will simply remove it.

Let's carry out these adjustments.

In [ ]:
# 1. Заменим подозрительное количество детей на NaN
df.children = df.children.replace( { -1:np.nan, 20:np.nan } )

# Заменим NaN на среднее количество детей в каждой категории income_type
df.children = df.children.fillna( df.groupby( 'income_type' ).children.transform( 'mean' ) )

# Убедимся, что все дети целочисленные. Мы сломали тип данных в этой колонке - починим в следующей секции
df.children = round( df.children )

# 2. Заменяем неродившихся в колонке dob_years на NaN
df.dob_years = df.dob_years.replace( 0, np.nan )

# Заменим NaN на средний возраст в каждой категории income_type
df.dob_years = df.dob_years.fillna( df.groupby( 'income_type' ).dob_years.transform( 'mean' ) )

# 3. Сделаем days_employed положительными
df.drop( 'days_employed', inplace=True, axis=1 )
columns_values.remove( 'days_employed' )

# Проверим успех
round( df[ columns_values ].describe(), 1 )

,children,dob_years,total_income
count,21525.0,21525.0,19351.0
mean,0.5,43.5,167422.3
std,0.8,12.2,102971.6
min,0.0,19.0,20667.3
25%,0.0,34.0,103053.2
50%,0.0,43.0,145017.9
75%,1.0,53.0,203435.1
max,5.0,75.0,2265604.0


Ура! Эти значения теперь выглядит достаточно прилично. Посмотрим на вторую часть этого упражнения.

In [ ]:
# Проверим уникальные значения в 'категориальные' колонках
for column in columns_category:
    print( 'Название колонки:', column )
    print( 'Список уникальных значений:' )
    display( raw_df[ column ].value_counts() )
    print( '-' * 50 )

Название колонки: education
Список уникальных значений:


среднее                13750
высшее                  4718
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   274
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
УЧЕНАЯ СТЕПЕНЬ             1
Ученая степень             1
Name: education, dtype: int64

--------------------------------------------------
Название колонки: education_id
Список уникальных значений:


1    15233
0     5260
2      744
3      282
4        6
Name: education_id, dtype: int64

--------------------------------------------------
Название колонки: family_status
Список уникальных значений:


...

--------------------------------------------------
Название колонки: debt
Список уникальных значений:


0    19784
1     1741
Name: debt, dtype: int64

--------------------------------------------------
Название колонки: purpose
Список уникальных значений:


свадьба                                   797
на проведение свадьбы                     777
сыграть свадьбу                           774
операции с недвижимостью                  676
покупка коммерческой недвижимости         664
операции с жильем                         653
покупка жилья для сдачи                   653
операции с коммерческой недвижимостью     651
покупка жилья                             647
жилье                                     647
покупка жилья для семьи                   641
строительство собственной недвижимости    635
недвижимость                              634
операции со своей недвижимостью           630
строительство жилой недвижимости          626
покупка недвижимости                      624
строительство недвижимости                620
покупка своего жилья                      620
ремонт жилью                              612
покупка жилой недвижимости                607
на покупку своего автомобиля              505
заняться высшим образованием      

--------------------------------------------------


Нам повезло: приключения нас ждут только в колонках `education` и `family_status` (разный регистр), `gender` (имеется странное значение `XNA`) и `purpose` (похожие по смыслу категории 'приобретение автомобиля' и 'покупка автомобиля'). Эти проблемы (кроме `XNA`) мы устраним в следующих секциях.

In [ ]:
# Уберем XNA из датасета
df = df[ df.gender != 'XNA' ]

### 2 - Отсутствующие значения в стобцах
Поехали дальше.

In [ ]:
# Посчитаем количество пропусков в данных
df.isna().sum()

children               0
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Как мы и предполагали - у нас есть пропуски в колонке `total_income`. Посмотрим какую долю от общего количества строк они составляют.

Количество пустых значений также равно 2174 - значит в одной строчке значение пропущено в двух столбцах. Тогда можем работать с пустыми значениями построчно.

Теперь проверим какую долю пустые строки занимают во всем датасете.

In [ ]:
# Поделим количество пропущенных строк в total_income на общее количесто строк (возьмем children как колонку со всеми значениями)
print( 'Доля пустых строк в датасете:', round( df.total_income.isna().sum() / df.children.count() * 100, 1), '%'  )
display( df[ df.total_income.isna() ].head() )

Доля пустых строк в датасете: 10.1 %


,children,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0.0,65.0,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0.0,41.0,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0.0,63.0,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0.0,50.0,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0.0,54.0,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу


Нам опять повезло: количество пропущенных значений относительно мало. Похоже, что они все случайны: вряд ли у нас был мужчина-госслужащий даже со средним образованием без опыта работы и без какого-либо заработка (см. строку 26). Поэтому, мы можем заменить эти значения средними в рамках одного типа занятости: такой подход несильно исказит общую картину, учитывая, что у нас может быть большой разбег в значениях дохода для разных групп.

In [ ]:
# Перезапишем колонку totol_income. Теперь вместо пустых значений будут стоять средние значения для каждой категории income_type
df.total_income = df.total_income.fillna( df.groupby('income_type').total_income.transform( 'mean' ) )

### 3 - Повторяющиеся строки, нестандратные названия категорий и типы данных в колонках

Пора посмотреть на возможные повторы. Т.к. у нас нету уникальных идентификаторов записи в датасете, то я подойду к проблеме напрямую: если есть строчки, где все значения одинаковые - будем считать это дубликатом.

В новом задании предлагается сначала стандартизировать регистр и типы данных. Я предлагаю сначала удалить дубликаты, а потом уже приступить к стандартизации: во-первых, `total_income` имеет много знаков после запятой (может, значение было переведено из одной валюты в другую) - этим можно повысить "уникальность" строки; во-вторых, если предположить, что разный регистр в `income_type` - это следствие ручного ввода данных человеком, то это тоже поможет избежать удаления лишних строк.

In [ ]:
# Посчитаем количество повторяющихся строк (без учета первой встретившейся строки)
print( 'Количество повторяющихся строк:', df.duplicated().sum() )

# Посмотрим на виноватых
display( df[ df.duplicated() ].sort_values( by=['income_type', 'dob_years'] ).head() )

Количество повторяющихся строк: 54


,children,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
18349,1.0,30.0,высшее,0,женат / замужем,0,F,госслужащий,0,170898.309923,покупка жилья для семьи
14432,2.0,36.0,высшее,0,женат / замужем,0,F,госслужащий,0,170898.309923,получение образования
13878,1.0,31.0,среднее,1,женат / замужем,0,F,компаньон,0,202417.136353,покупка жилья
19387,0.0,38.0,высшее,0,гражданский брак,1,F,компаньон,0,202417.136353,на проведение свадьбы
10697,0.0,40.0,среднее,1,гражданский брак,1,F,компаньон,0,202417.136353,сыграть свадьбу


Похоже, все повторяющиеся строки случайны: в этих заявках не были указаны значения `total_income` (поэтому они одинаковые - мы их заменили средними). Так как мы не значем что такое NaN в контексте этих данных (значение не указано, потому что пропустили или потому что оно равно нулю), то простым решением будет вырезать эти строки из датасета.

In [ ]:
# Уберем повторяющиеся строки - сделаем это до исправления регистра
df = df.drop_duplicates()

# Нижний регистр для образования и семейного положения
for column in ['education', 'family_status']:
    df[ column ] = df[ column ].str.lower()
    
# Зададим тип данных в колонках
df[ [ 'children', 'dob_years', 'total_income' ] ] = df[ [ 'children', 'dob_years', 'total_income' ] ].astype('int64')

# Посмотрим, что получилось
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21470 entries, 0 to 21524
Data columns (total 11 columns):
children            21470 non-null int64
dob_years           21470 non-null int64
education           21470 non-null object
education_id        21470 non-null int64
family_status       21470 non-null object
family_status_id    21470 non-null int64
gender              21470 non-null object
income_type         21470 non-null object
debt                21470 non-null int64
total_income        21470 non-null int64
purpose             21470 non-null object
dtypes: int64(6), object(5)
memory usage: 2.0+ MB


### 4 - Словари (lookup tables)

В датасете у нас есть две колонки, которые могут быть использованы как ключи - `education_id` и `family_status_id`. Выделим их и соответсвтующие словари в отдельные таблицы.

In [ ]:
# Словарь education
lut_education = df[ [ 'education_id', 'education' ] ].drop_duplicates().reset_index( drop=True )

# Словарь family_status
lut_family_status = df[ [ 'family_status_id', 'family_status' ] ].drop_duplicates().reset_index( drop=True )

### 5 - Классификация `total_income`

На основании диапазонов, указанных ниже, создадим столбец `total_income_category` с категориями:
- **E**: 0 – 30,000
- **D**: 30,001 – 50,000
- **C**: 50,001 – 200,000
- **B**: 200,001 – 1,000,000
- **A**: 1,000,001 и выше.

In [ ]:
# Функция для категоризации
def fun_categorizer_total_income( value_for_categorization ):
    """
    This function allocates a category for a variable based on its value. 

    Args:
        value_for_categorization ([float or int]): value for categorization

    Returns:
        - 'E' if value_for_categorization <= 30,000
        - 'D' if value_for_categorization = [30,001 : 50,000]
        - 'C' if value_for_categorization = [50,001 : 200,000]
        - 'B' if value_for_categorization = [200,001 : 1,000,000]
        - 'A' if value_for_categorization >= 1,000,001
    """
    if value_for_categorization <= 30e3:
        return 'E'
    elif 30e3 < value_for_categorization <= 50e3:
        return 'D'
    elif 50e3 < value_for_categorization <= 200e3:
        return 'C'
    elif 200e3 < value_for_categorization <= 1e6:
        return 'B'
    else:
        return 'A'

# Применим функцию к датасету
df[ 'total_income_category' ] = df.total_income.apply( fun_categorizer_total_income )

### 6 - Классификация `purpose`

Наконец, пора добавить стандартные категории к колонке `purpose`.

In [ ]:
# Заколхозим
def fun_categorizer_purpose( string_for_categorization ):
    """
    This function categorizes a string if a particular set of characters is present. 

    Args:
        string_for_categorization [str]: string, where a set of characters is checked

    Returns:
        - 'операции с автомобилем' if string contains 'авто'
        - 'операции с недвижимостью' if string contains 'нед' or 'жил'
        - 'проведение свадьбы' if string contains 'свад'
        - 'получение образования' if string contains 'образ'
    """
    if 'авто' in string_for_categorization:
        return 'операции с автомобилем'
    elif 'свад' in string_for_categorization:
        return 'проведение свадьбы'
    elif 'образ' in string_for_categorization:
        return 'получение образования'
    else:
        return 'операции с недвижимостью'

df[ 'purpose_category' ] = df.purpose.apply( fun_categorizer_purpose )

На этом можно закончить предобработку данных. Пора заняться исследованием!

## Проверка гипотез

Нам надо проверить 4 гипотезы:

1. Есть ли зависимость между наличием детей и возвратом кредита в срок?
2. Есть ли зависимость между семейным положением и возвратом кредита в срок?
3. Есть ли зависимость между уровнем дохода и возвратом кредита в срок?
4. Как разные цели кредита влияют на его возврат в срок?

Процесс проверки будет простым: для каждого из этих вопросов построим сводную таблицу. Строки таблицы будут соответствовать категориям, а столбцы - общему количеству людей в категории, количеству людей вернувших кредит в срок, количеству людей невернувших кредит в срок. На основе этих колонок вычислим соответсвтующие доли и потом сделаем вывод.

Скажем, если разница между категориями более 3% - это статистически значимо, а значит зависимость есть. В обртном случае - зависимости нет. Я не имею опыта в статистике, но подозреваю, что в серьезном упражнении 3% были бы более аргументированы.

In [ ]:
# Сделаем функцию, которая будет выдавать нам результаты
def fun_counter( data, rows ):
    # Сводная таблица #1 - влияние наличия детей
    pivot = pd.pivot_table( df, 
                            values='gender', 
                            index=rows, 
                            columns='debt', 
                            aggfunc='count', 
                            fill_value=0 )

    # Поправим названия колонок, чтобы легче было жить
    pivot = pivot.rename( columns={ 0:'people_without_debt_no', 1:'people_with_debt_no' }, level=0 )

    # Суммарное количество людей в категории
    pivot['people_total'] = pivot.people_without_debt_no + pivot.people_with_debt_no

    # Доля людей имеющих и не имеющих задолжность
    pivot['people_without_debt_per'] = pivot.people_without_debt_no / pivot.people_total
    pivot['people_with_debt_per'] = pivot.people_with_debt_no / pivot.people_total
      
    # Возвращаем табличку
    return pivot.sort_values( by=[ 'people_with_debt_per', 'people_total' ] )

### 1 - Зависимость между наличием детей и возвратом кредита в срок

Проверим, что получилось для первой гипотезы.

In [ ]:
display( fun_counter( df, 'children' ) )

debt,people_without_debt_no,people_with_debt_no,people_total,people_without_debt_per,people_with_debt_per
children,,,,,
5,9,0,9,1.000000,0.000000
0,13060,1063,14123,0.924733,0.075267
3,303,27,330,0.918182,0.081818
1,4462,453,4915,0.907833,0.092167
2,1858,194,2052,0.905458,0.094542
4,37,4,41,0.902439,0.097561


Первое, что бросается в глаза - не все категории достаточно большие, чтобы делать выводы. Так, у нас есть мало людей у кого есть 3, 4 и 5 детей, но сильно больше (почти в 10 раз) людей, у кого нет детей, есть 1 или 2 ребенка.

**Вывод:** Тем не менее, соагласно нашему критерию в 3% непохоже, чтобы была какая-то значимая зависимость между наличием детей и возвратом кредита в срок. Во всех категориях вероятность возврата состовляет около 90%

### 2 - Зависимость между семейным положением и возвратом кредита в срок

Проверим, что получилось для второй гипотезы.

In [ ]:
display( fun_counter( df, 'family_status' ) )

debt,people_without_debt_no,people_with_debt_no,people_total,people_without_debt_per,people_with_debt_per
family_status,,,,,
вдовец / вдова,896,63,959,0.934307,0.065693
в разводе,1110,85,1195,0.928870,0.071130
женат / замужем,11413,931,12344,0.924579,0.075421
гражданский брак,3774,388,4162,0.906776,0.093224
не женат / не замужем,2536,274,2810,0.902491,0.097509


В этом случае у нас более равномерная выборка (по сравнению с предыдущей гипотезой), поэтому посмотрим на все категории.

**Вывод:** В этом случае у нас чуть более выраженная зависимость. Гражданский брак и отсутствие жены / мужа явно снижает вероятность возврата кредита (хотя она все еще около 90%) по сравнению с остальными категориями (вероятность возврата около 93%). Интересно, что самая надежная категория - это вдовец / вдова, хотя, вероятно, это следствие относительно маленькой выборки.

### 3 - Зависимость между уровнем дохода и возвратом кредита в срок

Проверим, что получилось для третьей гипотезы.

In [ ]:
display( fun_counter( df, 'total_income_category' ) )

debt,people_without_debt_no,people_with_debt_no,people_total,people_without_debt_per,people_with_debt_per
total_income_category,,,,,
D,329,21,350,0.940000,0.060000
B,5158,386,5544,0.930375,0.069625
A,23,2,25,0.920000,0.080000
C,14199,1330,15529,0.914354,0.085646
E,20,2,22,0.909091,0.090909


Опять некоторые категгории получились очень маленькими - всего 25 людей в категории по доходу 'A' (что ожидаемо).

**Вывод:** В этом случае у нас тоже есть зависимость, хотя и не совсем, как я ожидал. Видно, что люди с наименьшим доходом имеют самую низкую вероятность возврата кредита (все еще около 90%). В то время как более высокий доход может значительно это вероятность повысить (до 93% в случае 'B'). Интересно, что первую строчку у нас все равно заняла категория 'D'.

### 4 - Зависимость между целями кредита и возвратом кредита в срок

Проверим, что получилось для последней гипотезы.

In [ ]:
display( fun_counter( df, 'purpose_category' ) )

debt,people_without_debt_no,people_with_debt_no,people_total,people_without_debt_per,people_with_debt_per
purpose_category,,,,,
операции с недвижимостью,10031,782,10813,0.927680,0.072320
проведение свадьбы,2149,186,2335,0.920343,0.079657
получение образования,3644,370,4014,0.907823,0.092177
операции с автомобилем,3905,403,4308,0.906453,0.093547


**Вывод:** А вот это неожиданный результат. Похоже, что кредиты на операции с недвижимостью и на проведение свадьбы возвращают реже, чем на получение образования и операции с автомобилем. Хотя разица оказалось не очень большой (всего ~2%), она заметна. Еще интересно, как каждые из двух категорий сгруппировались друг с другом.

## Выводы

В этом проекте я прошел весь цикл анализа данных: от загрузки до проверки гипотез. В первой части, я вычистил данные: привел к стандартному виду, разобрался с пустыми значениями, убрал дубликаты, сделал разные категории. Это была самая трудозатратная часть. Во второй части, я провел относительно простой анализ и сравнил вероятность возвращения кредита в срок для разных срезов в данных.

В итоге, мы получили, что количество детей слабо влияет на вероятность возврата кредита (хотя есть небольшая зависимость - чем больше детей, тем вероятность возврата ниже). Вероятно, это связано с дополнительными расходами на содержание ребенка.

Семейный статус оказывает заметное влияние: так люди, состоящие в зарегистрированных отношениях более чаще возвращают кредиты. Вероятно, это связно с некоторым "взрослением" которое сопровождает решение вступить в брак.

Уровень дохода также может оказать влияние на вероятность возврата кредита: как можно ожидать, чем меньше доход - тем (примерно) вероятность меньше. Исключение составляет категория 'D', которая оказалась самой надежной. Возможно, здесь бы следовало пересмотреть группировку по доходу: как-то неравномерно у нас распределились заявители.

Наконец, цели кредита тоже влияют на вероятность возврата. Удивительно, что кредит на свадьбу возвращают чаще, чем кредит на образование. Возможно, это также связано с тем, что в свадьбе (или приобретении жилья) могут участвовать два человека, тогда как кредиты на образование (или покупку недвижимости) берутся в одиночестве...